<a href="https://colab.research.google.com/github/Ricardo-Jaramillo/PySpark/blob/main/01_Spark_DataFrames_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operations with DataFrames

Now let's use some of the most common operations and methods to work with data

First we need to install and import PySpark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=533a3bc23974f71df9f00e4b4a5dc6c89292fe7ae4ba416663ab5f16c248634e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [6]:
# import the dataset we'll use
!wget https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/appl_stock.csv

--2023-10-02 23:53:32--  https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/appl_stock.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/plain]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K  --.-KB/s    in 0.03s   

2023-10-02 23:53:33 (4.63 MB/s) - ‘appl_stock.csv’ saved [143130/143130]



## Define a SparkSession

 This time we're gonna set a SparSession in which we are going to allocate our temporary views

In [7]:
from pyspark.sql import SparkSession

In [8]:
# Define the session
spark = SparkSession.builder.appName('ops').getOrCreate()

In [9]:
# Read in the data and print the schema
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)

In [10]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [11]:
# See the data
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [12]:
# See the first rows
df.head(3)[0]

Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

## Filtering data
Check some ways to filter our dataframe

In [13]:
# Filter the data with NO-SQL sintax, specifying as string condition
df.filter('Close < 500').select(['Open', 'close']).show()

+------------------+------------------+
|              Open|             close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [14]:
# Filter data specifying the column as df
df.filter(df['Close'] < 500).select('Volume').show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



### Operational sintax
* ~ -> not
* & -> and
* | -> or

all multiple conditions must be inside parenthesis

In [15]:
# Filter with multiple conditions
df.filter((df['close'] < 200) & ~(df['Open'] > 200)).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

## Show() vs Collect()

* `Show()` only let us see the data in a "prettier" way than just the raw data.
* `Collect()` returns a list with the values of our request.

In [16]:
# Using Show()
df.filter(df['Low'] == 197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



In [17]:
# Using Collect()
results = df.filter(df['Low'] == 197.16).collect()

In [18]:
# Print our results list
results

[Row(Date=datetime.date(2010, 1, 22), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [19]:
# Select the first element (the unique element for this example)
row = results[0]

In [20]:
# Return the results as a dict
row.asDict()

{'Date': datetime.date(2010, 1, 22),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [21]:
# Show a value by its dictionary key
row['Volume']

220441900

## GroupBy and Aggregate operations

In [25]:
# Download the file
!wget https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/sales_info.csv

--2023-10-02 23:57:37--  https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/sales_info.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196 [text/plain]
Saving to: ‘sales_info.csv’

sales_info.csv      100%[===================>]     196  --.-KB/s    in 0s      

2023-10-02 23:57:37 (10.0 MB/s) - ‘sales_info.csv’ saved [196/196]



In [22]:
# Define the session
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [26]:
# Read in the data
df = spark.read.csv('sales_info.csv', inferSchema=True,
                    header=True)

In [27]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [28]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [29]:
# Let's group it
df.groupBy('Company')

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

In [36]:
# Apply a grouping method
df.groupBy('Company').count().show()

df.groupBy('Company').mean().show()

df.groupBy('Company').max().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [40]:
# Apply an aggregate method to the whole data column
df.agg({'Sales': 'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [41]:
# Group the data and then apply an agg function to compare results
group_data = df.groupBy('Company')

In [42]:
# Get the sum of each grouping label
group_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



## Import some functions from the pyspark.sql library

In [43]:
# Import functions
from pyspark.sql.functions import countDistinct, avg, stddev

In [47]:
df.select(countDistinct('Sales')).show()

df.select(avg('Sales')).show()

df.select(stddev('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [49]:
# Change the column name as an alias
df.select(avg('Sales')).alias('Average Sales').show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



## Change the number of decimals to show

In [51]:
# First let's check the total number of decimals in a result
df.select(stddev('Sales')).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [52]:
# Import the necessary function
from pyspark.sql.functions import format_number

In [55]:
# Again check the aggregate function with an alias
sales_std = df.select(stddev('Sales').alias('std'))
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [59]:
# Reformat the number
sales_std.select(format_number('std', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [60]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



## Different ways to order data

In [71]:
# Order by column name
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [72]:
# Descendent order by column Object
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

